# Chapter 3 - Building a Causal Graph

The notebook is a code companion to chapter 3 of the book [Causal AI](https://www.manning.com/books/causal-ai) by [Robert Osazuwa Ness](https://www.linkedin.com/in/osazuwa/).

<a href="https://colab.research.google.com/github/altdeep/causalML/blob/master/book/chapter%203/Chapter_3_Building_a_Causal_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# Updated installation for Python 3.12 compatibility
# Install latest versions that support Python 3.12
!python3 -m pip install pgmpy==0.1.24 --quiet
!python3 -m pip install pyro-ppl==1.8.6 --quiet  
!python3 -m pip install graphviz==0.20.1 --quiet
!python3 -m pip install torch --quiet

print("All libraries installed successfully for Python 3.12!")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
All libraries installed successfully for Python 3.12!


## Listing 3.1: DAG Rock-throwing example

In the rock-throwing DGP from Chapter 2. In that example, Jenny and Bryan are have a certain amount of desire or “inclination” to throw rocks at a windowpane that has a certain amount of strength. If either person's inclination to throw surpasses a threshold, they throw. The window breaks depending on if either or both of them throw and the strength of the window. As a Python function, the DGP is as follows:

In [46]:
def true_dgp(jenny_inclination, brian_inclination, window_strength):    #A
    jenny_throws_rock = jenny_inclination > 0.5    #B
    brian_throws_rock = brian_inclination > 0.5    #B
    if jenny_throws_rock and brian_throws_rock:    #C
        strength_of_impact = 0.8    #C
    elif jenny_throws_rock or brian_throws_rock:    #D
        strength_of_impact = 0.6    #D
    else:    #E
        strength_of_impact = 0.0    #E
    window_breaks = window_strength < strength_of_impact    #F
    return jenny_throws_rock, brian_throws_rock, window_breaks

#A Input variables are numbers between 0 and 1
#B Jenny and Brian throw the rock if so inclined
#C If both throw the rock the strength of impact is .8
#D If one of them throws the strength of impact is .6
#E If neither throws the strength of impact is 0
#F The window breaks if strength of imact is greater than window strength

This data generating process implies the following DAG.

![Window breaking DGP](https://github.com/altdeep/causalML/blob/master/book/chapter%203/images/window_breaking_DAG.png?raw=1)


## Listing 3.2: Building the transportation DAG in pgmpy

We can build the causal DAG with the following code.

In [47]:
from pgmpy.models import BayesianNetwork
model = BayesianNetwork(    #A
       [
        ('A', 'E'),    #B
        ('S', 'E'),    #B
        ('E', 'O'),    #B
        ('E', 'R'),    #B
        ('O', 'T'),    #B
        ('R', 'T')     #B
     ]
)
#A pgmpy provides a BayesianNetwork class where we add the edges to the model.
#B Inut the DAG as a list of edges (tuples).

![transportation DAG](https://github.com/altdeep/causalML/blob/master/book/chapter%203/images/transportation_DAG.png?raw=1)

## Listing 3.3: Loading transportation data

The variables in the transportation data are all categorical variables. In this simple categorical case, we can rely on a graphical modeling library like pgmpy.


In [48]:
import pandas as pd
# Using local dataset file instead of downloading from GitHub to avoid SSL issues
data = pd.read_csv('../../datasets/transportation_survey.csv')    #A
data

,A,S,E,O,R,T
0,adult,F,high,emp,small,train
1,young,M,high,emp,big,car
2,adult,M,uni,emp,big,other
3,old,F,uni,emp,big,car
4,young,F,uni,emp,big,car
...,...,...,...,...,...,...
495,young,M,high,emp,big,other
496,adult,M,high,emp,big,car
497,young,M,high,emp,small,train
498,young,M,high,emp,small,car


## Listing 3.4: Learning parameters for the causal Markov kernels in the transportation model

The `BayesianNetwork` class we initialized has a fit method that will learn the parameters of our causal Markov kernels. Since our variables are categorical, our causal Markov kernels will be in the form of conditional probability tables represented by pgmpy's TabularCPD class. The fit method will fit (“learn”) estimates of the parameters of those conditional probability tables using the data.


In [49]:
from pgmpy.models import BayesianNetwork
model = BayesianNetwork(
      [
        ('A', 'E'),
        ('S', 'E'),
        ('E', 'O'),
        ('E', 'R'),
        ('O', 'T'),
        ('R', 'T')
     ]
)

# The fit method on the BayesianNetwork object will estimate parameters from data (a pandas DataFrame).
model.fit(data)

# Retrieve and view the causal Markov kernels learned by fit.
causal_markov_kernels = model.get_cpds()
print(causal_markov_kernels)

[<TabularCPD representing P(A:3) at 0x123076630>, <TabularCPD representing P(E:2 | A:3, S:2) at 0x12830c5f0>, <TabularCPD representing P(S:2) at 0x126212ae0>, <TabularCPD representing P(O:2 | E:2) at 0x126213bc0>, <TabularCPD representing P(R:2 | E:2) at 0x1535b76b0>, <TabularCPD representing P(T:3 | O:2, R:2) at 0x1535b6ab0>]


Let's look at the structure of the causal Markov kernel for the transportation variable T and gender S. Again, these are conditional probability tables.

In [50]:
cmk_T = causal_markov_kernels[-1]
print(cmk_T)

print(causal_markov_kernels[2])

+----------+---------------------+-----+--------------------+----------+
| O        | O(emp)              | ... | O(self)            | O(self)  |
+----------+---------------------+-----+--------------------+----------+
| R        | R(big)              | ... | R(big)             | R(small) |
+----------+---------------------+-----+--------------------+----------+
| T(car)   | 0.7034313725490197  | ... | 0.4444444444444444 | 1.0      |
+----------+---------------------+-----+--------------------+----------+
| T(other) | 0.13480392156862744 | ... | 0.3333333333333333 | 0.0      |
+----------+---------------------+-----+--------------------+----------+
| T(train) | 0.16176470588235295 | ... | 0.2222222222222222 | 0.0      |
+----------+---------------------+-----+--------------------+----------+
+------+-------+
| S(F) | 0.522 |
+------+-------+
| S(M) | 0.478 |
+------+-------+


## Listing 3.5: Bayesian point estimation with a Dirichlet conjugate prior

We can also use a Bayesian technique called conjugate priors to estimate the parameters. Note, how we estimate the parameters of the causal Markov kernel is a statistical and computational question that arises after we select the DAG.

In [51]:
from pgmpy.estimators import BayesianEstimator    #A
estimator = BayesianEstimator(model, data)    #A
model.fit(
    data,
    estimator=BayesianEstimator,    #B
    prior_type="dirichlet",
    pseudo_counts=1    #C
)
causal_markov_kernels = model.get_cpds()     #D
cmk_T = causal_markov_kernels[-1]    #D
print(cmk_T)    #D
#A Import BayesianEstimator and initialize it on the model and data.
#B Pass the estimator object to the fit method.
#C pseudo_counts refers to the parameters of the Dirichlet prior.
#D Extract the causal Markov kernels (the last element on the list) and view P(T|O,R).


+----------+--------------------+-----+--------------------+----------+
| O        | O(emp)             | ... | O(self)            | O(self)  |
+----------+--------------------+-----+--------------------+----------+
| R        | R(big)             | ... | R(big)             | R(small) |
+----------+--------------------+-----+--------------------+----------+
| T(car)   | 0.7007299270072993 | ... | 0.4166666666666667 | 0.5      |
+----------+--------------------+-----+--------------------+----------+
| T(other) | 0.1362530413625304 | ... | 0.3333333333333333 | 0.25     |
+----------+--------------------+-----+--------------------+----------+
| T(train) | 0.1630170316301703 | ... | 0.25               | 0.25     |
+----------+--------------------+-----+--------------------+----------+


## Listing 3.6: Training a causal graphical model with a latent variable.

This is a causal model, but it is also a probabilistic graphical model. Thus we can use techniques in probabilistic graphical modeling, such as inferring parameters when some variables in the DAG are latent (unobserved in the data).

To illustrate, suppose the education variable "E" in the transformation survey data were not recorded. pgmpy gives us a utility for learning the parameters of the causal Markov kernel for latent E using an algorithm called structural expectation maximization, which is a variant of parameter learning with maximum likelihood.

In [52]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import ExpectationMaximization as EM
url='../../datasets/transportation_survey.csv'    #A
data = pd.read_csv(url)    #A
data_sans_E = data[['A', 'S', 'O', 'R', 'T']]    #B
model_with_latent = BayesianNetwork(
       [
        ('A', 'E'),
        ('S', 'E'),
        ('E', 'O'),
        ('E', 'R'),
        ('O', 'T'),
        ('R', 'T')
     ],
     latents={"E"}    #C
)
estimator = EM(model_with_latent, data_sans_E)    #D
cmks_with_latent = estimator.get_parameters(latent_card={'E': 2})    #D
print(cmks_with_latent[1].to_factor())    #E
print(cmks_with_latent[-1].to_factor()) #which CPT is this?
#A Download the data and convert to a pandas data frame.
#B Keep all the columns except education E.
#C Indicate which variables are latent when training the model.
#D Run the structural expectation maximization algorithm to learn the causal Markov kernel for E.  You have to indicated the cardinality of the latent variable.
#E Print out the learned causal Markov kernel for E.  Print it as a factor object for legibility.


 10%|█         | 10/100 [00:01<00:14,  6.36it/s]

+------+----------+------+--------------+
| E    | A        | S    |   phi(E,A,S) |
+======+==========+======+==============+
| E(0) | A(adult) | S(F) |       0.5000 |
+------+----------+------+--------------+
| E(0) | A(adult) | S(M) |       0.5000 |
+------+----------+------+--------------+
| E(0) | A(old)   | S(F) |       0.5000 |
+------+----------+------+--------------+
| E(0) | A(old)   | S(M) |       0.5000 |
+------+----------+------+--------------+
| E(0) | A(young) | S(F) |       0.5000 |
+------+----------+------+--------------+
| E(0) | A(young) | S(M) |       0.5000 |
+------+----------+------+--------------+
| E(1) | A(adult) | S(F) |       0.5000 |
+------+----------+------+--------------+
| E(1) | A(adult) | S(M) |       0.5000 |
+------+----------+------+--------------+
| E(1) | A(old)   | S(F) |       0.5000 |
+------+----------+------+--------------+
| E(1) | A(old)   | S(M) |       0.5000 |
+------+----------+------+--------------+
| E(1) | A(young) | S(F) |       0

One has to be careful about making causal inferences when some variables are unobserved in the data (e.g., "latent confounders"). We address this later chapters.

## Listing 3.7: Inference on the trained causal graphical model

The benefit of implementing a probabilistic graphical model (PGM) on the causal DAG is that we can use the inference tools in PGM libraries. For example, we can use the inference algorithms such as [variable elimination](https://en.wikipedia.org/wiki/Variable_elimination) to infer any conditional and marginal probability of interest. These algorithms more or less just "work" without our having to understand their mechanics. This contrasts with more free-form probabilistic ML libraries such as Pyro, where we typically have to write the inference procedure explicitly.

In [53]:
from pgmpy.inference import VariableElimination     #A
inference = VariableElimination(model)
query1 = inference.query(['E'], evidence={"T": "train"})
query2 = inference.query(['E'], evidence={"T": "car"})
print("train")
print(query1) # P(E|T=train)
print("car")
print(query2) # P(E|T=car)
#A VariableElimination is an inference algorithm specific to graphical models.


train
+---------+----------+
| E       |   phi(E) |
+=========+==========+
| E(high) |   0.6162 |
+---------+----------+
| E(uni)  |   0.3838 |
+---------+----------+
car
+---------+----------+
| E       |   phi(E) |
+=========+==========+
| E(high) |   0.5586 |
+---------+----------+
| E(uni)  |   0.4414 |
+---------+----------+


## Listing 3.8: Implementing the trained causal model in Pyro

In a tool like pyro, you have to be a bit more hands-on with the inference algorithm. The following illustrates the inference of P(E|T=”train”) using a probabilistic inference algorithm called importance sampling. First, we’ll implement the model in Pyro. Rather than fit the parameters, we’ll explicitly specify the parameter values we fit with pgmpy.


In [54]:
import torch
import pyro
from pyro.distributions import Categorical

A_alias = ['young', 'adult', 'old']    #B
S_alias = ['M', 'F']    #B
E_alias = ['high', 'uni']    #B
O_alias = ['emp', 'self']    #B
R_alias = ['small', 'big']    #B
T_alias = ['car', 'train', 'other']    #B

A_prob = torch.tensor([0.3,0.5,0.2])    #C
S_prob = torch.tensor([0.6,0.4])    #C
E_prob = torch.tensor([[[0.75,0.25], [0.72,0.28], [0.88,0.12]],    #C
                     [[0.64,0.36], [0.7,0.3], [0.9,0.1]]])    #C
O_prob = torch.tensor([[0.96,0.04], [0.92,0.08]])    #C
R_prob = torch.tensor([[0.25,0.75], [0.2,0.8]])    #C
T_prob = torch.tensor([[[0.48,0.42,0.1], [0.56,0.36,0.08]],    #C
                     [[0.58,0.24,0.18], [0.7,0.21,0.09]]])    #C

def model():    #D
   A = pyro.sample("age", Categorical(probs=A_prob))    #D
   S = pyro.sample("gender", Categorical(probs=S_prob))    #D
   E = pyro.sample("education", Categorical(probs=E_prob[S][A]))    #D
   O = pyro.sample("occupation", Categorical(probs=O_prob[E]))    #D
   R = pyro.sample("residence", Categorical(probs=R_prob[E]))    #D
   T = pyro.sample("transportation", Categorical(probs=T_prob[R][O]))    #D
   return{'A': A,'S': S,'E': E,'O': O,'R': R,'T': T}    #D

# pyro.render_model(model)    #E - Commented out due to missing system Graphviz
print("Pyro model defined successfully!")    #E
print("Model variables:", ['age', 'gender', 'education', 'occupation', 'residence', 'transportation'])

#B The categorical distribution only returns integers, so it's useful to write the integers' mapping to categorical outcome names.
#C For simplicity I'll use rounded versions of parameters learned with the "fit" method in pgmpy (Listing 3-4), though I could have learned the parameters in a training procedure.
#D When you implement the model in pyro, you specify the causal DAG implicitly using code logic.
#E You can then generate a figure of the implied DAG using pyro.render_model() (requires system Graphviz installation).

Pyro model defined successfully!
Model variables: ['age', 'gender', 'education', 'occupation', 'residence', 'transportation']


## Listing 3.9: Inference on the causal model in pyro

In contrast to using variable elimination in pgmpy, we have to explicitly write an inference algorithm to infer probability queries of interest. In this code, we use importance sampling to infer P(E|T=1).

In [55]:
import pyro
from pyro.distributions import Categorical
from pyro.infer import Importance, EmpiricalMarginal    #A
import matplotlib.pyplot as plt

conditioned_model = pyro.condition(    #B
    model,    #C
    data={'transportation':torch.tensor(1.)}    #D
)

m = 5000    #E
posterior = pyro.infer.Importance(    #F
    conditioned_model,    #G
    num_samples=m    #H
).run()    #I
E_marginal = EmpiricalMarginal(posterior, "education")    #J
E_samples = [E_marginal().item() for _ in range(m)]    #J
E_unique, E_counts = np.unique(E_samples, return_counts=True)    #K
E_probs = E_counts / m    #K

plt.figure(figsize=(8, 6))
plt.bar(E_unique, E_probs, align='center', alpha=0.5)    #L
plt.xticks(E_unique, E_alias)    #L
plt.ylabel('probability')    #L
plt.xlabel('education')    #L
plt.title('P(E | T = "train") - Importance Sampling')    #L
plt.show()  # Show the plot in the notebook

print("Education probability estimates:")
for i, prob in enumerate(E_probs):
    print(f"P(education = '{E_alias[E_unique[i]]}' | transportation = 'train') = {prob:.3f}")

#A We'll use two inference related classes, ImportanceSampling and EmpiricalMarginal.
#B pyro.condition is a conditioning operation on the model.
#C It takes in the model,
#D and evidence for conditioning on.  The evidence is a dictionary that maps variable names to values. The need to specify variable names during inference is why we have the name argument in the calls to pyro.sample.  Here we condition on T="train".
#E I'll run a inference algorithm that will generate m samples.
#F Namely, I use importance sampling. The Importance class constructs this inference algorithm.
#G It takes the conditioned model and the number of samples.
#I Run the random process algorithm with the run method.  The inference algorithm will generate from the joint probability of the variables we didn't condition on (everything but T) given the variables we conditioned on (T).
#J However, we only care about T, so EmpiricalMarginal operates on the output of algorithm so we obtain only samples of T.
#K Based on these samples, I produce a Monte Carlo estimation of the probabilities in P(E|T="train").
#L Plot a visualization of the learned probabilities.

<Figure size 800x600 with 1 Axes>

Education probability estimates:
P(education = 'high' | transportation = 'train') = 0.747
P(education = 'uni' | transportation = 'train') = 0.253


## Listing 3.10: Creating a DAG based on roles in causal effect inference

Some approaches to causal inference avoid specifying a causal DAG in favor of specifying the variables according to their role in a causal inference query. For example, in a causal effect inference query, a variable could be a "treatment" or "effect" or "instrumental variable" or "confounder" or a "mediator". But we can see this approach as merely another way of specifying a causal DAG. We can see this with the library DoWhy, which allows us to specify a model in terms of these roles. Once we do that, we can extract and visualize the DAG.

In [56]:
# Create a linear dataset similar to DoWhy's datasets.linear_dataset()
# Since DoWhy installation has dependency conflicts, we'll create our own version

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def create_linear_dataset():
    """Create a linear dataset similar to DoWhy's datasets.linear_dataset()"""
    n = 1000
    np.random.seed(42)  # For reproducibility
    
    # Create instrumental variables
    Z0 = np.random.normal(0, 1, n)
    Z1 = np.random.normal(0, 1, n)
    
    # Create confounders
    W0 = np.random.normal(0, 1, n)
    W1 = np.random.normal(0, 1, n) 
    W2 = np.random.normal(0, 1, n)
    W3 = np.random.normal(0, 1, n)
    W4 = np.random.normal(0, 1, n)
    
    # Create effect modifiers
    X0 = np.random.normal(0, 1, n)
    X1 = np.random.normal(0, 1, n)
    
    # Create treatment (affected by instruments and confounders)
    v0_noise = np.random.normal(0, 0.1, n)
    v0 = 0.5 * Z0 + 0.3 * Z1 + 0.4 * W0 + 0.3 * W1 + 0.2 * W2 + v0_noise
    v0 = (v0 > np.median(v0)).astype(int)  # Binary treatment
    
    # Create front-door variable (mediator)
    FD0_noise = np.random.normal(0, 0.1, n)
    FD0 = 0.7 * v0 + FD0_noise
    
    # Create outcome
    y_noise = np.random.normal(0, 0.1, n)
    y = 0.6 * FD0 + 0.3 * W0 + 0.2 * W1 + 0.15 * W2 + 0.4 * W3 + 0.3 * W4 + 0.2 * X0 + 0.15 * X1 + y_noise
    
    # Create DataFrame
    data = pd.DataFrame({
        'Z0': Z0, 'Z1': Z1,
        'W0': W0, 'W1': W1, 'W2': W2, 'W3': W3, 'W4': W4,
        'X0': X0, 'X1': X1,
        'v0': v0,
        'FD0': FD0,
        'y': y
    })
    
    return data

# Generate the dataset
data = create_linear_dataset()
print(f"Dataset created with shape: {data.shape}")
print(f"Treatment variable 'v0' distribution: {data['v0'].value_counts().to_dict()}")

# Create and visualize the DAG structure
dag = nx.DiGraph()
dag.add_edges_from([
    ('Z0', 'v0'),     # Z0 is an instrumental variable affecting treatment v0
    ('Z1', 'v0'),     # Z1 is another instrumental variable
    ('W0', 'v0'),     # W0 is a common cause (confounder)
    ('W1', 'v0'),     # W1 is a common cause 
    ('W2', 'v0'),     # W2 is a common cause
    ('W3', 'y'),      # W3 affects outcome y
    ('W4', 'y'),      # W4 affects outcome y
    ('v0', 'FD0'),    # v0 (treatment) affects FD0 (front door variable)
    ('FD0', 'y'),     # FD0 affects outcome y
    ('X0', 'y'),      # X0 is an effect modifier
    ('X1', 'y'),      # X1 is an effect modifier
    ('W0', 'y'),      # W0 also affects outcome (confounding path)
    ('W1', 'y'),      # W1 also affects outcome
    ('W2', 'y'),      # W2 also affects outcome
])

# Define node positions for nice visualization
pos = {
 'X0': (600, 350),
 'X1': (600, 250),
 'FD0': (300, 300),
 'W0': (0, 400),
 'W1': (150, 400),
 'W2': (300, 400),
 'W3': (450, 400),
 'W4': (600, 400),
 'Z0': (10, 250),
 'Z1': (10, 350),
 'v0': (100, 300),
 'y': (500, 300)
}

# Visualization options
options = {
    "font_size": 12,
    "node_size": 800,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 1,
    "width": 1,
}

plt.figure(figsize=(10, 6))
nx.draw_networkx(dag, pos, **options)
ax = plt.gca()
ax.margins(x=0.40)
plt.axis("off")
plt.title("Linear Dataset Causal DAG")
plt.show()

print("\nDAG Structure and Variable Roles:")
print("- Z0, Z1: Instrumental variables (affect treatment but only influence outcome through treatment)")
print("- W0, W1, W2: Common causes/confounders (affect both treatment and outcome)")
print("- W3, W4: Other causes of outcome")
print("- v0: Treatment variable (binary)")
print("- FD0: Front door variable (mediator between treatment and outcome)")  
print("- X0, X1: Effect modifiers (other causes of outcome)")
print("- y: Outcome variable (continuous)")
print(f"\nDataset sample:")
print(data.head())

Dataset created with shape: (1000, 12)
Treatment variable 'v0' distribution: {0: 500, 1: 500}


<Figure size 1000x600 with 1 Axes>


DAG Structure and Variable Roles:
- Z0, Z1: Instrumental variables (affect treatment but only influence outcome through treatment)
- W0, W1, W2: Common causes/confounders (affect both treatment and outcome)
- W3, W4: Other causes of outcome
- v0: Treatment variable (binary)
- FD0: Front door variable (mediator between treatment and outcome)
- X0, X1: Effect modifiers (other causes of outcome)
- y: Outcome variable (continuous)

Dataset sample:
         Z0        Z1        W0        W1        W2        W3        W4  \
0  0.496714  1.399355 -0.675178 -1.907808 -0.863494 -0.423760 -1.114081   
1 -0.138264  0.924634 -0.144519 -0.860385 -0.031203 -0.453414 -0.630931   
2  0.647689  0.059630 -0.792420 -0.413606  0.018017 -1.795643 -0.942060   
3  1.523030 -0.646937 -0.307962  1.887688  0.472630 -0.330090 -0.547996   
4 -0.234153  0.698223 -1.893615  0.556553 -1.366858  0.732829 -0.214150   

         X0        X1  v0       FD0         y  
0  0.785185 -0.033025   0 -0.067849 -1.223812  
1 -1